In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import os, shutil
from glob import glob
import requests
from bs4 import BeautifulSoup

In [2]:
# selects scenes

bounds = gpd.read_file('4FRI.gpkg')
bounds = bounds.to_crs('EPSG:4269')
 
WRS_PATH = 'Landsat8/WRS2_descending_0.zip'
LANDSAT_PATH = os.path.dirname(WRS_PATH)

shutil.unpack_archive(WRS_PATH, os.path.join(LANDSAT_PATH, 'wrs2'))

wrs = gpd.GeoDataFrame.from_file('Landsat8/wrs2/WRS2_descending.shp')
wrs_intersection0 = wrs[wrs.intersects(bounds.geometry[0])]
wrs_intersection1 = wrs[wrs.intersects(bounds.geometry[1])]
wrs_intersection2 = wrs[wrs.intersects(bounds.geometry[2])]
wrs_intersection3 = wrs[wrs.intersects(bounds.geometry[3])]

pieces = [wrs_intersection0, wrs_intersection1, wrs_intersection2, wrs_intersection3]

wrs_intersection = pd.DataFrame(columns=wrs_intersection0.columns.tolist())

for piece in pieces: 
    wrs_intersection = wrs_intersection.append(piece)

wrs_intersection = wrs_intersection.drop_duplicates()

first_delete = wrs_intersection.loc[(wrs_intersection['PATH'] == 38) & (wrs_intersection['ROW'] == 36)]
second_delete = wrs_intersection.loc[(wrs_intersection['PATH'] == 36) & (wrs_intersection['ROW'] == 35)]

wrs_intersection = wrs_intersection[~wrs_intersection.isin(first_delete)]
wrs_intersection = wrs_intersection[~wrs_intersection.isin(second_delete)]

wrs_intersection = wrs_intersection.dropna()

print(wrs_intersection)

paths, rows = wrs_intersection['PATH'].values, wrs_intersection['ROW'].values

xy = np.asarray(bounds.centroid[0].xy).squeeze()
center = list(xy[::-1])

zoom = 6

m = folium.Map(location=center, zoom_start=zoom, control_scale=True)

m.add_child(folium.GeoJson(bounds.__geo_interface__, name='4FRI Footprint', 
                           style_function=lambda x: {'color': 'red', 'alpha': 0}))

for i, row in wrs_intersection.iterrows():
    name = 'path: %03d, row: %03d' % (row.PATH, row.ROW)
    g = folium.GeoJson(row.geometry.__geo_interface__, name=name)
    g.add_child(folium.Popup(name))
    g.add_to(m)

folium.LayerControl().add_to(m)
m.save('wrs.html')

m

scene_selection = pd.read_csv('index.csv', chunksize=1000)

bulk_list = []

for chunk in scene_selection: 
    
    for path, row in zip(paths, rows):
        
        scenes = chunk[(chunk.WRS_PATH == path) & (chunk.WRS_ROW == row) & 
                           (chunk.CLOUD_COVER <= 5) & 
                           (~chunk.PRODUCT_ID.str.contains('_T2')) &
                           (~chunk.PRODUCT_ID.str.contains('_RT')) &
                           (chunk.SPACECRAFT_ID == 'LANDSAT_8') & 
                           (chunk.SENSING_TIME > '2020-01-01') & 
                           (chunk.SENSING_TIME < '2020-06-01')]

        if len(scenes):
            scenes = scenes.sort_values('CLOUD_COVER').iloc[0]

        if len(scenes) > 0:
            bulk_list.append(scenes)
            
print(len(bulk_list))

          AREA  PERIMETER    PR_  PR_ID RINGS_OK RINGS_NOK PATH ROW MODE  \
2019   3.11508    7.14451  532.0  532.0        1         0   36  36    D   
2020   3.06357    7.07561  533.0  533.0        1         0   36  37    D   
14667  3.11508    7.14451  532.0  532.0        1         0   37  36    D   
14666  3.16838    7.21602  531.0  531.0        1         0   37  35    D   
27314  3.16838    7.21602  531.0  531.0        1         0   38  35    D   
18263  3.11598    7.14578  532.0  532.0        1         0   35  36    D   
18264  3.06508    7.07759  533.0  533.0        1         0   35  37    D   

      SEQUENCE   WRSPR      PR ACQDayL7 ACQDayL8  \
2019      6236  036036  036036        2       10   
2020      6237  036037  036037        2       10   
14667    31532  037036  037036        9        1   
14666    31531  037035  037035        9        1   
27314    56827  038035  038035       16        8   
18263    38724  035036  035036       11        3   
18264    38725  035037  035

<ipython-input-2-134dbd49bc9b>:38: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  xy = np.asarray(bounds.centroid[0].xy).squeeze()


30


In [3]:
# downloads scene selection

bulk_frame = pd.concat(bulk_list, 1).T
bulk_frame = bulk_frame.sort_values(by=['DATE_ACQUIRED'], ascending=False)
bulk_frame.to_csv('scene_selection.csv')
bulk_frame = pd.read_csv('scene_selection.csv')

print(bulk_frame.head())
print(len(bulk_frame))

for i, row in bulk_frame.iterrows():
    
    print('\n', 'EntityId:', row.PRODUCT_ID, '\n')
    print(' Checking content: ', '\n')
    
    row_url = 'https://landsat-pds.s3.amazonaws.com/c1/L8/0{}/0{}/{}/index.html'.format(row.WRS_PATH, row.WRS_ROW, row.PRODUCT_ID)
    
    response = requests.get(row_url)

    html = BeautifulSoup(response.content, 'html.parser')
    entity_dir = os.path.join(LANDSAT_PATH, row.PRODUCT_ID)
    
    if os.path.isdir(entity_dir) == False: 
        
        os.makedirs(entity_dir, exist_ok=True)
        
        for li in html.find_all('li'):

            file = li.find_next('a').get('href')

            print('  Downloading: {}'.format(file))

            response = requests.get(row_url.replace('index.html', file), stream=True)

            with open(os.path.join(entity_dir, file), 'wb') as output:
                shutil.copyfileobj(response.raw, output)
            del response

   Unnamed: 0               SCENE_ID  \
0     2003360  LC80360372020150LGN00   
1     9999084  LC80360362020150LGN00   
2    10431066  LC80380352020148LGN00   
3    10187947  LC80350362020143LGN00   
4     1991175  LC80350372020143LGN00   

                                 PRODUCT_ID SPACECRAFT_ID SENSOR_ID  \
0  LC08_L1TP_036037_20200529_20200608_01_T1     LANDSAT_8  OLI_TIRS   
1  LC08_L1TP_036036_20200529_20200608_01_T1     LANDSAT_8  OLI_TIRS   
2  LC08_L1TP_038035_20200527_20200608_01_T1     LANDSAT_8  OLI_TIRS   
3  LC08_L1TP_035036_20200522_20200607_01_T1     LANDSAT_8  OLI_TIRS   
4  LC08_L1TP_035037_20200522_20200607_01_T1     LANDSAT_8  OLI_TIRS   

  DATE_ACQUIRED  COLLECTION_NUMBER COLLECTION_CATEGORY  \
0    2020-05-29                  1                  T1   
1    2020-05-29                  1                  T1   
2    2020-05-27                  1                  T1   
3    2020-05-22                  1                  T1   
4    2020-05-22                  1        

  Downloading: LC08_L1TP_035036_20200404_20200410_01_T1_B9_wrk.IMD
  Downloading: LC08_L1TP_035036_20200404_20200410_01_T1_B11.TIF.ovr
  Downloading: LC08_L1TP_035036_20200404_20200410_01_T1_B1.TIF.ovr
  Downloading: LC08_L1TP_035036_20200404_20200410_01_T1_B5.TIF.ovr
  Downloading: LC08_L1TP_035036_20200404_20200410_01_T1_B9.TIF

 EntityId: LC08_L1TP_035037_20200404_20200410_01_T1 

 Checking content:  

  Downloading: LC08_L1TP_035037_20200404_20200410_01_T1_B6.TIF
  Downloading: LC08_L1TP_035037_20200404_20200410_01_T1_B1_wrk.IMD
  Downloading: LC08_L1TP_035037_20200404_20200410_01_T1_B1.TIF
  Downloading: LC08_L1TP_035037_20200404_20200410_01_T1_B7.TIF
  Downloading: LC08_L1TP_035037_20200404_20200410_01_T1_ANG.txt
  Downloading: LC08_L1TP_035037_20200404_20200410_01_T1_B7_wrk.IMD
  Downloading: LC08_L1TP_035037_20200404_20200410_01_T1_B1.TIF.ovr
  Downloading: LC08_L1TP_035037_20200404_20200410_01_T1_B3.TIF
  Downloading: LC08_L1TP_035037_20200404_20200410_01_T1_BQA_wrk.IMD
  Down

  Downloading: LC08_L1TP_038035_20200324_20200409_01_T1_B9.TIF
  Downloading: LC08_L1TP_038035_20200324_20200409_01_T1_B8.TIF
  Downloading: LC08_L1TP_038035_20200324_20200409_01_T1_B3.TIF.ovr
  Downloading: LC08_L1TP_038035_20200324_20200409_01_T1_B2_wrk.IMD
  Downloading: LC08_L1TP_038035_20200324_20200409_01_T1_B4.TIF.ovr
  Downloading: LC08_L1TP_038035_20200324_20200409_01_T1_B10.TIF
  Downloading: LC08_L1TP_038035_20200324_20200409_01_T1_B6.TIF
  Downloading: LC08_L1TP_038035_20200324_20200409_01_T1_ANG.txt
  Downloading: LC08_L1TP_038035_20200324_20200409_01_T1_B11.TIF.ovr
  Downloading: LC08_L1TP_038035_20200324_20200409_01_T1_B5_wrk.IMD
  Downloading: LC08_L1TP_038035_20200324_20200409_01_T1_BQA.TIF.ovr
  Downloading: LC08_L1TP_038035_20200324_20200409_01_T1_B7.TIF
  Downloading: LC08_L1TP_038035_20200324_20200409_01_T1_B2.TIF.ovr
  Downloading: LC08_L1TP_038035_20200324_20200409_01_T1_B1.TIF
  Downloading: LC08_L1TP_038035_20200324_20200409_01_T1_B1.TIF.ovr
  Downloading: LC08

  Downloading: LC08_L1TP_035037_20200131_20200211_01_T1_B10.TIF
  Downloading: LC08_L1TP_035037_20200131_20200211_01_T1_B3_wrk.IMD
  Downloading: LC08_L1TP_035037_20200131_20200211_01_T1_B4_wrk.IMD
  Downloading: LC08_L1TP_035037_20200131_20200211_01_T1_B6_wrk.IMD
  Downloading: LC08_L1TP_035037_20200131_20200211_01_T1_BQA.TIF.ovr
  Downloading: LC08_L1TP_035037_20200131_20200211_01_T1_B5.TIF.ovr
  Downloading: LC08_L1TP_035037_20200131_20200211_01_T1_B9.TIF.ovr
  Downloading: LC08_L1TP_035037_20200131_20200211_01_T1_B10.TIF.ovr
  Downloading: LC08_L1TP_035037_20200131_20200211_01_T1_ANG.txt
  Downloading: LC08_L1TP_035037_20200131_20200211_01_T1_B6.TIF.ovr
  Downloading: LC08_L1TP_035037_20200131_20200211_01_T1_B7.TIF.ovr
  Downloading: LC08_L1TP_035037_20200131_20200211_01_T1_B9.TIF
  Downloading: LC08_L1TP_035037_20200131_20200211_01_T1_BQA.TIF
  Downloading: LC08_L1TP_035037_20200131_20200211_01_T1_B2.TIF
  Downloading: LC08_L1TP_035037_20200131_20200211_01_T1_B11.TIF
  Downloading

  Downloading: LC08_L1TP_037035_20200113_20200127_01_T1_B8_wrk.IMD
  Downloading: LC08_L1TP_037035_20200113_20200127_01_T1_B6.TIF
  Downloading: LC08_L1TP_037035_20200113_20200127_01_T1_B5_wrk.IMD
  Downloading: LC08_L1TP_037035_20200113_20200127_01_T1_B6_wrk.IMD
  Downloading: LC08_L1TP_037035_20200113_20200127_01_T1_B11.TIF.ovr
  Downloading: LC08_L1TP_037035_20200113_20200127_01_T1_B6.TIF.ovr
  Downloading: LC08_L1TP_037035_20200113_20200127_01_T1_B10.TIF.ovr
  Downloading: LC08_L1TP_037035_20200113_20200127_01_T1_B10_wrk.IMD
  Downloading: LC08_L1TP_037035_20200113_20200127_01_T1_BQA.TIF
  Downloading: LC08_L1TP_037035_20200113_20200127_01_T1_B5.TIF.ovr
  Downloading: LC08_L1TP_037035_20200113_20200127_01_T1_B4.TIF.ovr
  Downloading: LC08_L1TP_037035_20200113_20200127_01_T1_B1.TIF.ovr
  Downloading: LC08_L1TP_037035_20200113_20200127_01_T1_B3.TIF.ovr
  Downloading: LC08_L1TP_037035_20200113_20200127_01_T1_B7.TIF
  Downloading: LC08_L1TP_037035_20200113_20200127_01_T1_B3_wrk.IMD
  D

In [ ]:
# performs atmospheric correction on scenes

